In [1]:
# 모듈 import

from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.utils import get_device

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

# config file 들고오기
cfg = Config.fromfile('./configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py')

root='../../../../../dataset/'

# dataset config 수정
cfg.data.train.classes = classes
cfg.data.train.img_prefix = root
cfg.data.train.ann_file = root + 'train.json' # train json 정보
cfg.data.train.pipeline[2]['img_scale'] = (512,512) # Resize

cfg.data.test.classes = classes
cfg.data.test.img_prefix = root
cfg.data.test.ann_file = root + 'test.json' # test json 정보
cfg.data.test.pipeline[1]['img_scale'] = (512,512) # Resize

cfg.data.samples_per_gpu = 4

cfg.seed = 20
cfg.gpu_ids = [0]
cfg.work_dir = './work_dirs/faster_rcnn_r50_fpn_1x_trash'

cfg.model.roi_head.bbox_head.num_classes = 10

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.checkpoint_config = dict(max_keep_ckpts=3, interval=1)
cfg.device = get_device()

In [6]:
cfg.log_config.hooks = [
    dict(type='TextLoggerHook'),
    dict(type='MMDetWandbHook',
         init_kwargs={'project': 'object-detection'},
         interval=10,
         log_checkpoint=True,
         log_checkpoint_metadata=True,
         num_eval_images=100)]


In [7]:
# build_dataset
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=0.09s)
creating index...
index created!


In [4]:
# dataset 확인
datasets[0]


CocoDataset Train dataset with number of images 4883, and instance counts: 
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| category          | count | category      | count | category        | count | category    | count | category     | count |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| 0 [General trash] | 3965  | 1 [Paper]     | 6352  | 2 [Paper pack]  | 897   | 3 [Metal]   | 936   | 4 [Glass]    | 982   |
| 5 [Plastic]       | 2943  | 6 [Styrofoam] | 1263  | 7 [Plastic bag] | 5178  | 8 [Battery] | 159   | 9 [Clothing] | 468   |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+

In [8]:
# 모델 build 및 pretrained network 불러오기
model = build_detector(cfg.model)
model.init_weights()

2024-10-14 15:40:34,387 - mmcv - INFO - initialize ResNet with init_cfg {'type': 'Pretrained', 'checkpoint': 'torchvision://resnet50'}
2024-10-14 15:40:34,388 - mmcv - INFO - load model from: torchvision://resnet50
2024-10-14 15:40:34,390 - mmcv - INFO - load checkpoint from torchvision path: torchvision://resnet50
2024-10-14 15:40:34,509 - mmcv - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias

2024-10-14 15:40:34,542 - mmcv - INFO - initialize FPN with init_cfg {'type': 'Xavier', 'layer': 'Conv2d', 'distribution': 'uniform'}
2024-10-14 15:40:34,568 - mmcv - INFO - initialize RPNHead with init_cfg {'type': 'Normal', 'layer': 'Conv2d', 'std': 0.01}
2024-10-14 15:40:34,574 - mmcv - INFO - initialize Shared2FCBBoxHead with init_cfg [{'type': 'Normal', 'std': 0.01, 'override': {'name': 'fc_cls'}}, {'type': 'Normal', 'std': 0.001, 'override': {'name': 'fc_reg'}}, {'type': 'Xavier', 'distribution': 'uniform', 'override'

In [9]:
# 모델 학습
train_detector(model, datasets[0], cfg, distributed=False, validate=False)

2024-10-14 15:40:36,740 - mmdet - INFO - Automatic scaling of learning rate (LR) has been disabled.
2024-10-14 15:40:37,170 - mmdet - INFO - Start running, host: root@instance-12243, work_dir: /data/ephemeral/home/KSG-git/level2-objectdetection-cv-20/code/baseline/mmdetection/work_dirs/faster_rcnn_r50_fpn_1x_trash
2024-10-14 15:40:37,171 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) MMDetWandbHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) NumClassCheckHook                  
(LOW         ) IterTimerHook                      
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) MMDetWandbHook                     
 -------------------- 
before_train_iter:
(VERY_HIGH   ) StepLrUpdaterHook        

2024-10-14 15:40:38,820 - mmdet - WARNING - No meta information found in the runner. 
2024-10-14 15:40:38,821 - mmdet - WARNING - To log evaluation or checkpoint metadata in MMDetWandbHook, `EvalHook` or `DistEvalHook` in mmdet is required, please check whether the validation is enabled.
2024-10-14 15:40:52,665 - mmdet - INFO - Epoch [1][50/1221]	lr: 1.978e-03, eta: 1:07:21, time: 0.277, data_time: 0.051, memory: 2099, loss_rpn_cls: 0.5064, loss_rpn_bbox: 0.0652, loss_cls: 0.6341, acc: 86.7812, loss_bbox: 0.1188, loss: 1.3245, grad_norm: 7.1782
2024-10-14 15:41:02,735 - mmdet - INFO - Epoch [1][100/1221]	lr: 3.976e-03, eta: 0:57:59, time: 0.201, data_time: 0.007, memory: 2099, loss_rpn_cls: 0.1578, loss_rpn_bbox: 0.0510, loss_cls: 0.3241, acc: 93.7422, loss_bbox: 0.2393, loss: 0.7722, grad_norm: 2.7395
2024-10-14 15:41:12,634 - mmdet - INFO - Epoch [1][150/1221]	lr: 5.974e-03, eta: 0:54:28, time: 0.198, data_time: 0.007, memory: 2099, loss_rpn_cls: 0.1201, loss_rpn_bbox: 0.0377, loss_c

KeyboardInterrupt: 

wandb: WARNING Fatal error while uploading data. Some run data will not be synced, but it will still be written to disk. Use `wandb sync` at the end of the run to try uploading.
